In [1]:
import sys
import os
from pathlib import Path
import re
import numpy as np
import pickle

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

COMMON = Path('../data/raw/COMMON_images_masks/')
GROUP = Path('../data/raw/GROUP_images/')
DATA = (COMMON.parent).parent
MASKS = DATA/'masks'

import SimpleITK as sitk
from utils import *
from linear_registration import register_linear

In [2]:
common_names = ['common_40', 'common_41', 'common_42']
common_paths = [COMMON/(name+'_image.nii.gz') for name in common_names]
group_paths = [file for file in GROUP.iterdir()]
group_pattern = re.compile(r'^(g\d{1,2}_\d{2})_image\.nii\.gz$')

group_names = []

for file in GROUP.iterdir():
    match = group_pattern.match(file.name)
    if match:
        group_names.append(match.group(1))

print(f"{common_names=}")
print(f"{group_names=}")


common_names=['common_40', 'common_41', 'common_42']
group_names=['g3_60', 'g1_54', 'g5_66', 'g8_74', 'g4_64', 'g11_85', 'g11_83', 'g3_59', 'g10_80', 'g1_55', 'g6_68', 'g6_69', 'g12_86', 'g2_58', 'g10_82', 'g8_75', 'g4_63', 'g4_62', 'g9_78', 'g5_65', 'g7_73', 'g2_57', 'g9_79', 'g9_77', 'g7_72', 'g1_56', 'g10_81', 'g7_71']


In [3]:
# COMMON images manual classification
# Create or load the training vectors for group images
pkl_path = DATA/'training/common_train_vects.pkl'
if os.path.exists(pkl_path):
    with open(pkl_path, "rb") as f:
        common_train_vects = pickle.load(f)
    print("common_train_vects loaded from file")

else:
    common_train_vects = []

    for path in common_paths:
        im = sitk.ReadImage(path)
        n_slices = im.GetSize()[2]
        common_train_vects.append(np.zeros(n_slices, dtype=int))
        del im

    with open(pkl_path, "wb") as f:
        pickle.dump(common_train_vects, f)

    print("common_train_vects created and saved")

common_train_vects loaded from file


In [6]:

# 2. Iterar sobre cada array en la lista para encontrar los rangos
for i, arr in enumerate(common_train_vects):
    # np.where devuelve una tupla, tomamos el primer elemento [0]
    indices_unos = np.where(arr == 1)[0]

    if indices_unos.size > 0:
        idx_inicio = indices_unos.min()
        idx_fin = indices_unos.max()
        cantidad = indices_unos.size
        
        print(f"Array {i}: Rango de índices con valor 1 -> [{idx_inicio} : {idx_fin}] (Total: {cantidad} elementos)")
    else:
        print(f"Array {i}: No contiene ningún valor 1 (todo son ceros).")

# except FileNotFoundError:
#     print(f"No se encontró el archivo '{nombre_archivo}'. Verifica la ruta.")
# except Exception as e:
#     print(f"Ocurrió un error: {e}")

Array 0: Rango de índices con valor 1 -> [23 : 51] (Total: 29 elementos)
Array 1: Rango de índices con valor 1 -> [43 : 65] (Total: 23 elementos)
Array 2: Rango de índices con valor 1 -> [22 : 48] (Total: 27 elementos)
